### Part1: From extracting opcode To making opcode image

In [ ]:
import os
import pickle
import pandas as pd

import re
import numpy as np
import hashlib as h

from skimage import io
#from ttictoc import TicToc
from nltk.util import ngrams

from utility.processing import make_opcode_lookup_table, make_line_start_lookup_table, is_code, make_hash_768

##### 1. pre-processing .asm files to image

In [ ]:
file_path = 'D:/virus/asm_data/'                     # load path for data in asm format
save_path_for_opcode = 'D:/virus/opcode/'            # save path for the extracted opcodes
save_path_for_image ='D:/virus/image/1gram_768/'     # save path for the image: opcode -> image
path_for_opcode = './utility/opcode_set.csv'
path_for_code_section = './utility/code_section.xlsx'
empty = set()

op_lookup_table = make_opcode_lookup_table(path_for_opcode)
start_text = make_line_start_lookup_table(path_for_code_section)    # Customizing 
start_text.add('.tixt')
start_text.add('.icode')
start_text.add('seg')

no_opcode_file = []
file_list = os.listdir(file_path)

##### 2. Extracting opcode sequence from the file_list and save it

In [ ]:
check = lambda x: True if not start_text[0] in x and not start_text[1] in x else False

for num, file in enumerate(file_list):
    file_opcode = []

    for line in open(file_path+file,'r',encoding='ISO-8859-1'):
        if not is_code(line, start_text):
            continue

        words = line.split()
        words = set(words)
        opcode = words.intersection(op_lookup_table)

        if opcode == empty:
            continue

        else:
            file_opcode.append(opcode.pop())
            continue

    if file_opcode == []:
        no_opcode_file.append(file)
        print(f"{file} doesn't have any opcode in CODE or TEXT section")
        continue

    file_name = file.replace('.asm', '.pkl')

    with open(save_path_for_opcode+file_name, 'wb') as fp:
        pickle.dump(file_opcode, fp)

    if num% 1000 == 0:
        print('Now processing the {}th file'.format(num+1))

In [ ]:
bits1 = 256
bits2 = 512
bits = [bits1 + bits2, bits1, bits2]

width = 24
height = 32

start_time = time.time()
make_img_768(bits, width, height, save_path_for_image)  # default: 1garm

print(f'Took: {time.time() - start_time}s seconds')

.

### Part2: From opcode image To converted image

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
%matplotlib notebook

RS = 123        # random state variable

from PIL import Image
from tqdm import tqdm
from sklearn.decomposition import PCA

from itertools import cycle
from statsmodels.graphics.factorplots import interaction_plot

##### 1. Loading image data

In [ ]:
gram_list = ['1gram_768/', '2gram_768/', '3gram_768/', '4gram_768/']
path = 'D:/virus/image/'

image_list = [[],[],[],[]]
label_list = []

gram_index = 0

for gram in tqdm(gram_list):
    PATH = path+gram
    image_file_list = os.listdir(PATH)
    
    for file_name in image_file_list:
        PATH2 = PATH + file_name
        
        if file_name[-3:]=='npy':
            image = np.load(PATH2).reshape(-1)/255.0
        
        else:
            image = np.array(Image.open(PATH2)).reshape(-1)/255.0
            label = int(file_name[-5]) -1

        image_list[gram_index].append(image)
        label_list.append(label)

    gram_index += 1
    
for i in range(3):
    image_list[i] = np.asarray(image_list[i])
    
y = np.asarray(label_list[:10736])

x_1 = image_list[0]
x_2 = image_list[1]
x_3 = image_list[2]
x_4 = image_list[3]

##### 2. Transforming image by PCA

In [ ]:
%%time
# 1-gram
selected_n_gram = x_1
clf_pca = PCA()
clf_pca.fit(selected_n_gram)
data1 = clf_pca.fit_transform(selected_n_gram)

# 2-gram
selected_n_gram = x_2
clf_pca = PCA()
clf_pca.fit(selected_n_gram)
data2 = clf_pca.fit_transform(selected_n_gram)

# 3-gram
selected_n_gram = x_3
clf_pca = PCA()
clf_pca.fit(selected_n_gram)
data3 = clf_pca.fit_transform(selected_n_gram)

# 4-gram
selected_n_gram = x_4
clf_pca = PCA()
clf_pca.fit(selected_n_gram)
data4 = clf_pca.fit_transform(selected_n_gram)

##### 3. Saving converted image: opcode image -> pca transformed image

In [ ]:
# 1gram img data
np.save(file = 'D:/virus/image/1gram_768_pca/image_arr.npy', arr = data1)
np.save(file = 'D:/virus/image/1gram_768_pca/label_arr.npy', arr = y )

# 2gram img data
np.save(file = 'D:/virus/image/2gram_768_pca/image_arr.npy', arr = data2)
np.save(file = 'D:/virus/image/2gram_768_pca/label_arr.npy', arr = y )

# 3gram img data
np.save(file = 'D:/virus/image/3gram_768_pca/image_arr.npy', arr = data3)
np.save(file = 'D:/virus/image/3gram_768_pca/label_arr.npy', arr = y )

# 4gram img data
np.save(file = 'D:/virus/image/4gram_768_pca/image_arr.npy', arr = data4)
np.save(file = 'D:/virus/image/4gram_768_pca/label_arr.npy', arr = y )